In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Import usefull liabraries

import numpy as np # linear algebra
import pandas as pd # data processing, data manupulation
import seaborn as sns # data visualization
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style

#import models (Algorithms)
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# import the dataset both train and test data
train_df = pd.read_csv('../input/titanic-dataset/train.csv')
test_df = pd.read_csv('../input/titanic-dataset/test.csv')


In [21]:
# combine the train and test data to one list
df = [train_df, test_df]

In [5]:
# printing the first 5 rows of the train_df data
# head command will output the first 5 rows of the dataset as default, if you want more rows then 5, you can specifically mention on the brackets like train_df.head(10)
train_df.head()

In [6]:
# printing the first 5 rows of the test_df data
test_df.head()

In [7]:
# print the last 5 rows of the train_df
# tail command will output the last 5 rows of the dataset as default, if you want more rows then 5, you can specifically mention on the brackets like train_df.tail(10)
train_df.tail()

In [8]:
# print the last 5 rows of the test_df
test_df.tail()

In [9]:
# check the shape of the dataset, this can help you to understand how many rows and columns in the dataset
print(f"the train data has {train_df.shape[0]} rows and {train_df.shape[1]} columns")
print(f"the test data has {test_df.shape[0]} rows and {test_df.shape[1]} columns") # Note: test data doesn't contain the target column

In [10]:
# pandas 'info()' command will give you a more information about data, like each column datatype, non-null values count etc.
train_df.info()
print('-'*30)
test_df.info()

In [11]:
# check the Null or Missing values in each column of train data
train_df.isnull().sum()

In train data we have some Missing or Null values on Age,Cabin and Embarked columns, we will fill these Null values using some methods in data cleaning

In [12]:
# check the Null or Missing values in each column of test data
test_df.isnull().sum()

In test data we have some Missing or Null values on Age,Fare and Cabin columns, we will fill these Null values using some methods in data preprocessing

In [13]:
# check the survival rate of pclass
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [14]:
#visual representation of survival rate per Pclass
sns.barplot(x='Pclass',y='Survived',data=train_df)

We can clearly see the above barplot has higher Survival rate on Pclass-1 

In [15]:
#survival rate based on sex. Female=0 and Male=1
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [16]:
#Visual representation of survival rate
sns.barplot(x='Sex',y='Survived',data=train_df)

In the above barplot Female are the higer survival rate then Male

# Data Cleaning

Now that we can have an idea of what types of data are in the dataset and what data is missing on dataset, we can clean or wrangling the data. We mainly focus on transform the string data to Numerical data and we can fill Missing or Null values with some methods.If any column is not usefull for Prediction we will drop that columns. and finally we transform all data to Numerical and Continuous because Machine can understand only Numerical data.

In [22]:
# we can convert Sex Column into Numerical values, Female=1, Male=0
# we can clean the data both train and test data, we already combined train and test data into one dataset(df) we will use this dataset to cleaning the data
for data in df:
    data['Sex']=data['Sex'].map({'female':1, 'male':0}).astype(int)

In [25]:
# check the Sex column in train and test data if the values are affected or not
print(train_df.head())
print('-'*30)
print(test_df.head())

In [32]:
# compute mean and standard deviation of Age column
age_mean = train_df['Age'].mean()
age_std = train_df['Age'].std()

# Number of NaN or Missing values on Age Column
num_na = train_df['Age'].isnull().sum()

# Now we can generate Random ages from mean and Standard deviation
random_vals = age_mean + age_std * np.random.randn(num_na)


In [33]:
# Now we can replace Age NaN values with random_vals
train_df.loc[train_df['Age'].isna(), 'Age'] = random_vals

# Ages are replaced with Continues values, Now we can convert into Whole Number
train_df['Age'] = train_df['Age'].astype(np.int64)

In [34]:
# check the train_df and test_df dataset[Age] if the values are affected are not
train_df.head()

In [37]:
# checking the Null values on Age column
print(train_df['Age'].isnull().sum())

In [38]:
# compute mean and standard deviation of Age column test_dataset
age_mean = test_df['Age'].mean()
age_std = test_df['Age'].std()

# Number of NaN or Missing values on Age Column
num_na = test_df['Age'].isnull().sum()

# Now we can generate Random ages from mean and Standard deviation
random_vals = age_mean + age_std * np.random.randn(num_na)

In [39]:
# Now we can replace Age NaN values with random_vals
test_df.loc[test_df['Age'].isna(), 'Age'] = random_vals

# Ages are replaced with Continues values, Now we can convert into Whole Number
test_df['Age'] = test_df['Age'].astype(np.int64)

In [42]:
# check the dataset if the values are affected are not
test_df['Age'].isnull().sum()

In [45]:
grid = sns.FacetGrid(train_df, row= 'Pclass', col= 'Sex', size = 2.4, aspect = 2.6)
grid.map(plt.hist, 'Age', alpha= .4, bins=10, color= 'blue')
plt.ylim((0,80))
grid.add_legend()

In [46]:
# find the most frequently used port
port = train_df['Embarked'].dropna().mode()[0]
port

In [48]:
# fill the Embarked column with most frequently used port
for data in df:
    data['Embarked'] = data['Embarked'].fillna(port)

# check the Survival rate on Embarked Column
train_df[['Embarked', 'Survived']].groupby(['Embarked'], 
                                as_index=False).mean().sort_values(by='Survived', ascending=False)

In [49]:
# check the train_df and test_df for any other missing values are remain
print(train_df.isnull().sum())
print('-'*30)
print(test_df.isnull().sum())

In [50]:
for data in df:
    data['Fare'] = data['Fare'].fillna(0)
    data['Fare'] = data['Fare'].astype(int)

In [ ]:
# convert categorical embarked feature to numeric values

for data in df:
    data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

# check the dataset to verify Embarked column is affected are not.
train_df.head()

In [54]:
train_df.head()

In [56]:
# Now we can drop some columns which are not usefull to predictions
# the columns we are droping here is passenderID, Name, Ticket, Cabin

train_df = train_df.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
test_df = test_df.drop(['Name','Ticket','Cabin'], axis=1)

In [57]:
print(train_df.shape)
print(test_df.shape)

In [58]:
age_hist = sns.FacetGrid(train_df, col= 'Survived')
age_hist.map(plt.hist, 'Age', bins = 20, color = "Orange")

Throughout the cleaning process I noticed a few relationships between age and survival. Most passangers in this data set are in the age range 15-35 years old. The oldest surviving passanger was 80 years old and children under the age of 4 had a high survival rate

In [61]:
# Survival by Age and Passendger Class
grid = sns.FacetGrid(train_df, col = 'Survived', row = 'Pclass', size = 2.3, aspect = 1.5)
grid.map(plt.hist, 'Age', alpha = .5, bins = 25, color = "red")
grid.add_legend();

In [62]:
# Survival by Sex, Passenger class and port 
grid = sns.FacetGrid(train_df, row='Embarked', size=2.3, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

# Prediction with Some Models

1. Logistic Regression
2. k-Nearest Neighbors
3. Random Fores
4. Decision Tree

In [65]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()

## Logistic Regression
Logistic regression is a statisical model used to handle classification problems. Logistic regression is a process of modeling the probablity of a discrete outcome given an input variable. In other words it measures the realatoinship between the categorical depedent feature and one of more independent features

In [76]:
# Survival Prediction using LogisticRegression

log_model = LogisticRegression()
log_model.fit(X_train, Y_train)
Y_pred = log_model.predict(X_test)
accuracy_log = round(log_model.score(X_train, Y_train) * 100, 2)
print(f" accuracy score with Random Forest : {accuracy_log}")

## Decision Tree

Decision Trees are a non-parametric surpervised learning method used for classifications and regression.The goal is to use a tree like model to evaluate decisions and their possible outcomes including things such as probablity, cost, and other relavent features. Decision tree models

In [71]:
# Survival Prediction using Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
print(f" accuracy score with Random Forest : {acc_decision_tree}")

## Random Forest
The random forest analysis is a classification algorithm consisting of many decision trees. However is utilizes a bagging method and randomness features.

In [70]:
# Survival Prediction using Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
print(f" accuracy score with Random Forest : {acc_random_forest}")

## K-Nearest Neighbor

The K-nearest Neighbor algorithm is a data classification method for estimating the likelihood of the data point will beocome a member of one group or another based on what the group data points nearest to is belong to.

In [73]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
print(f" accuracy score with Random Forest : {acc_knn}")

# Each Model performance

In [79]:
# Create a DataFrame with all models Accuracy Score with Model
models = pd.DataFrame({
   'Model': ['KNN', 'Logistic Regression', 'Random Forest',  'Decision Tree'],
    'Score': [ acc_knn,accuracy_log, acc_random_forest, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)